## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [2]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

In [ ]:


google_api_key = os.getenv('GOOGLE_API_KEY')


if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")



In [9]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [ ]:
messages

In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"
response=gemini.chat.completions.create(
    model=model_name,
    messages=messages
)
question=response.choices[0].message.content
print(question)


In [12]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

## Note - update since the videos

I've updated the model names to use the latest models below, like GPT 5 and Claude Sonnet 4.5. It's worth noting that these models can be quite slow - like 1-2 minutes - but they do a great job! Feel free to switch them for faster models if you'd prefer, like the ones I use in the video.

In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [15]:
# So where are we?

print(competitors)
print(answers)


['gemini-2.5-flash', 'llama3.2']
['The aspiration for Artificial Intelligence to achieve \'human-level intelligence\' is a powerful, almost mythical goal that has driven AI research for decades. Often referred to as Artificial General Intelligence (AGI), it posits that AI should be able to perform any intellectual task that a human can, with similar flexibility, learning capacity, and understanding. While seemingly a logical benchmark, a critical examination reveals a complex web of implicit assumptions and inherent limitations.\n\n### Implicit Assumptions About the Nature of Intelligence Itself:\n\nThe pursuit of human-level intelligence in AI makes several deep-seated, often unexamined, assumptions about what intelligence is:\n\n1.  **Intelligence is Singular and Measurable on a Linear Scale:** This goal assumes that intelligence is a monolithic entity that can be quantified and compared directly between humans and machines, much like an IQ score. It overlooks the concept of multiple

In [ ]:
# It's nice to know how to use "zip"
for competitor,answer in zip(competitors,answers):
    print(f"competitor: {competitor}\n\n{answer}\n")

In [20]:
# Let's bring this together - note the use of "enumerate"

together=""
for index, answer in enumerate(answers):
    together+=f"The response from competitor {index+1}\n\n"
    together+=f"{answer}\n\n"

In [ ]:
print(together)

In [22]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [24]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"
response=gemini.chat.completions.create(
    model=model_name,
    messages=judge_messages
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>

In [28]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
google_api_key=os.getenv("GOOGLE_API_KEY")
if google_api_key:
    print(f"Yes it exists and it starts with {google_api_key[:2]}")
else:
    print("Api Key Does not exist")

In [30]:
ollama=OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"
)

gemini=OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key
)

In [31]:
question = "How do u think Beginners need to approach to learn Agentic AI.. Is it Possible for them directly Jump in to learn agentic ai Without ML Knowledge"
messages=[{"role":"user","content":question}]


In [ ]:
llama_response=ollama.chat.completions.create(
    model="llama3.2",
    messages=messages

)
llama_answer=llama_response.choices[0].message.content
display(Markdown(llama_answer))

In [33]:
verify_prompt = f"""
You are a strict fact-checking and logic-verification agent.

Given:
QUESTION:
{question}

ANSWER:
{llama_answer}

Tasks:
1. Check factual correctness
2. Check logical consistency
3. Identify any hallucinations or errors
4. Output your verdict as JSON only

Format strictly as:
{{
  "is_correct": true/false,
  "issues": ["issue1", "issue2"],
  "confidence": 0.0-1.0
}}
"""


In [ ]:
gemini_message=[{"role":"user", "content":verify_prompt}]
gemini_response=gemini.chat.completions.create(
    model="gemini-2.0-flash",
    messages=gemini_message
)
gemini_answer=gemini_response.choices[0].message.content
print(gemini_answer)


In [ ]:
verification = json.loads(gemini_answer)

is_correct = verification["is_correct"]
issues = verification["issues"]
confidence = verification["confidence"]
print("\n FINAL VERDICT")
print("Correct:", is_correct)
print("Confidence:", confidence)

if not is_correct:
    print("\n⚠️ Issues found:")
    for issue in issues:
        print("-", issue)

